<a href="https://colab.research.google.com/github/vonvolous/d2l/blob/main/code/D2l_8_AlexNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torchsummary import summary
from torchvision import transforms, datasets

AlexNet 모델 정의
* 학습 데이터셋으론 FashionMNIST 사용
* 입력 채널 수가 grey scale이므로 1
* out_features는 데이터셋의 class가 10개이므로 10

In [3]:
class AlexNet(nn.Module):
    def __init__(self):
        super(AlexNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 96, kernel_size=11, stride=4)
        self.conv2 = nn.Conv2d(96, 256, kernel_size=5, padding=2)
        self.conv3 = nn.Conv2d(256, 384, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(384, 384, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(384, 256, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(256 * 6 * 6, 4096)
        self.fc2 = nn.Linear(4096, 4096)
        self.fc3 = nn.Linear(4096, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)

        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)

        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.max_pool2d(x, kernel_size=3, stride=2)

        # 4차원을 1차원으로 펼쳐줌 => flatten
        x = x.view(x.size(0), -1)

        x = F.relu(self.fc1(x))
        x = F.dropout(x, p=0.5)

        x = F.relu(self.fc2(x))
        x = F.dropout(x, p=0.5)

        x = F.log_softmax(self.fc3(x), dim=1)
        return x

# @staticmethod는 이 아래 정의된 함수를 클래스에 대한 객체 작업을 거치지 않고 클래스 외부에서 사용할 수 있게 해줌!
    @staticmethod
    def transform():
        return transforms.Compose([
            transforms.Resize((227, 227)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.1307,), std=(0.3081,))
        ])


model summary

In [5]:
if __name__ == "__main__":
    use_cuda = torch.cuda.is_available()
    print("use_cuda: ", use_cuda)

    FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
    device = torch.device("cuda:0" if use_cuda else "cpu")

    net = AlexNet().to(device)

    X = torch.randn(size=(1, 1, 227, 227)).type(FloatTensor)
    print(net(X))
    print(summary(net, (1, 227, 227)))

use_cuda:  True
tensor([[-2.3178, -2.2899, -2.3271, -2.3043, -2.2789, -2.3178, -2.2982, -2.3066,
         -2.2857, -2.3008]], device='cuda:0', grad_fn=<LogSoftmaxBackward0>)
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 55, 55]          11,712
            Conv2d-2          [-1, 256, 27, 27]         614,656
            Conv2d-3          [-1, 384, 13, 13]         885,120
            Conv2d-4          [-1, 384, 13, 13]       1,327,488
            Conv2d-5          [-1, 256, 13, 13]         884,992
            Linear-6                 [-1, 4096]      37,752,832
            Linear-7                 [-1, 4096]      16,781,312
            Linear-8                   [-1, 10]          40,970
Total params: 58,299,082
Trainable params: 58,299,082
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.20
Forward/backward pas

model train and test

In [8]:
if __name__ == "__main__":
    # hyper parameters
    batch_size = 512
    num_epochs = 3
    learning_rate = 0.0001

    # dataload
    root = './MNIST_Fashion'
    transform = AlexNet.transform()
    train_set = datasets.FashionMNIST(root=root,
                                      train=True,
                                      download=True,
                                      transform=transform)
    train_loader = torch.utils.data.DataLoader(train_set,
                                               batch_size=batch_size,
                                               shuffle=True)
    test_set = datasets.FashionMNIST(root=root,
                                     train=False,
                                     download=True,
                                     transform=transform)
    test_loader = torch.utils.data.DataLoader(test_set,
                                              batch_size=batch_size,
                                              shuffle=True)

    # gpu
    use_cuda = torch.cuda.is_available()
    print("use_cuda: ", use_cuda)
    device = torch.device("cuda:0" if use_cuda else "cpu")
    model = AlexNet().to(device)
    criterion = F.nll_loss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    def train(model, device, train_loader, optimizer, epoch):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            target = target.type(torch.LongTensor)
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            if (batch_idx + 1) % 30 == 0:
                print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                    100. * batch_idx / len(train_loader), loss.item()
                ))
    def test(model, device, test_loader):
        model.eval()
        test_loss = 0
        correct = 0
        with torch.no_grad():
            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                test_loss += criterion(output, target, reduction='sum').item()
                pred = output.max(1, keepdim=True)[1]
                correct += pred.eq(target.view_as(pred)).sum().item()

            test_loss /= len(test_loader.dataset)
            print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n".format(
                test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)
            ))
            print('=' * 50)

use_cuda:  True


train & test해서 accuracy & loss check

In [9]:
for epoch in range(1, num_epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [14848/60000 (25%)]	Loss: 0.860316
Train Epoch: 1 [30208/60000 (50%)]	Loss: 0.696333
Train Epoch: 1 [45568/60000 (75%)]	Loss: 0.536629

Test set: Average loss: 0.5245, Accuracy: 8069/10000 (81%)

Train Epoch: 2 [14848/60000 (25%)]	Loss: 0.419143
Train Epoch: 2 [30208/60000 (50%)]	Loss: 0.423421
Train Epoch: 2 [45568/60000 (75%)]	Loss: 0.403743

Test set: Average loss: 0.3873, Accuracy: 8577/10000 (86%)

Train Epoch: 3 [14848/60000 (25%)]	Loss: 0.384199
Train Epoch: 3 [30208/60000 (50%)]	Loss: 0.330158
Train Epoch: 3 [45568/60000 (75%)]	Loss: 0.344702

Test set: Average loss: 0.3515, Accuracy: 8671/10000 (87%)

